In [1]:
import requests 
import numpy as np

In [2]:
drivers=[]

url = 'http://ergast.com/api/f1/2018/drivers.json'
r = requests.get(url)

results = r.json()

for driver in results['MRData']['DriverTable']['Drivers']:
    drivers.append(str(driver['driverId']))

data = {}

for driver in drivers:

    url = 'http://ergast.com/api/f1/2018/drivers/'+driver+'/results.json'
    r = requests.get(url)

    results = r.json()

    races = []
    raceindex = []
    points = []

    for race in results['MRData']['RaceTable']['Races']:
        races.append(race['raceName'])
        raceindex.append(int(race['round']))
        points.append(int(race['Results'][0]['points']))


    points_cumsum = np.cumsum(points)
    
    data[driver] = {'index':raceindex,'race':races,'points':points,'cumulative_points':points_cumsum}
    
data

{'alonso': {'cumulative_points': array([10, 16, 22, 28, 32, 32, 32, 32, 36, 40, 40, 44], dtype=int32),
  'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
  'points': [10, 6, 6, 6, 4, 0, 0, 0, 4, 4, 0, 4],
  'race': ['Australian Grand Prix',
   'Bahrain Grand Prix',
   'Chinese Grand Prix',
   'Azerbaijan Grand Prix',
   'Spanish Grand Prix',
   'Monaco Grand Prix',
   'Canadian Grand Prix',
   'French Grand Prix',
   'Austrian Grand Prix',
   'British Grand Prix',
   'German Grand Prix',
   'Hungarian Grand Prix']},
 'bottas': {'cumulative_points': array([  4,  22,  40,  40,  58,  68,  86,  92,  92, 104, 122, 132],
        dtype=int32),
  'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
  'points': [4, 18, 18, 0, 18, 10, 18, 6, 0, 12, 18, 10],
  'race': ['Australian Grand Prix',
   'Bahrain Grand Prix',
   'Chinese Grand Prix',
   'Azerbaijan Grand Prix',
   'Spanish Grand Prix',
   'Monaco Grand Prix',
   'Canadian Grand Prix',
   'French Grand Prix',
   'Austrian Grand Prix',
   'B

In [6]:
from bokeh.io import output_notebook, show
from bokeh.layouts import row
from bokeh.palettes import Viridis256
from bokeh.plotting import figure
from bokeh.models import CheckboxGroup, CustomJS, HoverTool, ColumnDataSource
import bokeh
import math

output_notebook()



p = figure()
props = dict(line_width=4, line_alpha=0.7)
callback_string = '{}.visible = {} in checkbox.active;'
code_string = ''
i = 0
sources = []
labels = []
active = []
glyph_dict = {}
colors = bokeh.palettes.viridis(len(data)) # bokeh has been imported as bp for this to work


for driver in data:
    labels.append(driver)
    x = data[driver]['index']
    y = data[driver]['cumulative_points']
    gp = data[driver]['race']
    source =ColumnDataSource(data = {'x':x,'y':y, 'gp':gp})
    sources.append(source)
    line = p.line('x', 'y', source=sources[i], legend=driver, color=colors[i], **props)
    hover = HoverTool(tooltips=[('driver', driver),('points', '@y'),('gp','@gp')], renderers=[line])
    p.add_tools(hover)
    line.name = driver
    code_string += callback_string.format(driver, str(i))
    glyph_dict.update({driver:line})
    active.append(i)
    i += 1

checkbox = CheckboxGroup(labels=labels, active=active, width=200)
glyph_dict.update({'checkbox':checkbox})
checkbox.callback = CustomJS.from_coffeescript(args=glyph_dict, code=code_string)


p.legend.location = "top_left"
labels_override = dict(zip(data['bottas']['index'], data['bottas']['race']))
p.xaxis.major_label_overrides = labels_override
p.xaxis.major_label_orientation = math.pi/4


layout = row(checkbox, p)
show(layout)

Loading BokehJS ...